# Folds

Polars is built on top of a columnar data structure.  When you need to do a complex row-wise aggregation, the `folds` can be useful (in addition to the `axis=1` built-in aggregations to e.g. `sum`, `min`, etc).

In [1]:
import polars as pl
from polars import col

In [2]:
df = pl.DataFrame({
    "a": [1, 2, 3],
    "b": [10, 20, 30],
})

A manual sum:

In [5]:
df.select(
    pl.fold(acc=pl.lit(0), f=lambda acc, x: acc + x, exprs=col("*")).alias("sum")
)  # Q: that lambda is a python object -- isn't that slow?

sum
i64
11
22
33


But the docs say that the above operates on columns individually, and can take advantage of cache efficiency and vectorization... I guess the `+` is on two `Series` which is fast?

## Conditional

In [6]:
df = pl.DataFrame({
    "a": [1, 2, 3],
    "b": [0, 1, 2],
})

Give all rows where all columns per row are > 1

In [7]:
df.filter(
    pl.fold(
        acc = pl.lit(True),
        f = lambda acc, x: acc & x,
        exprs=col("*") > 1,
    )
)

a,b
i64,i64
3,2
